In [7]:
from torchtext import data
from torchtext.vocab import Vectors
from torch.nn import init
from tqdm import tqdm
import numpy as np
import os
from random import shuffle
import pandas as pd
import torch
import time
import random

In [8]:
Text = []
tag = [72, 71, 84, 54, 57, 65, 64]
Y = np.zeros(467)
Y[0 : tag[0]] = 0
Y[tag[0] : tag[0]+tag[1]] = 1
Y[tag[0]+tag[1] : tag[0]+tag[1]+tag[2]] = 2
Y[tag[0]+tag[1]+tag[2] : tag[0]+tag[1]+tag[2]+tag[3]] = 3
Y[tag[0]+tag[1]+tag[2]+tag[3] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]] = 4
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]] = 5
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]+tag[6]] = 6
Y = Y.tolist()
with open('/Users/denhiroshi/Downloads/Embeding/sentence.txt') as f:
    Text = f.readlines()
Text = Text[1:]
len(Text),len(Y)

(467, 467)

In [5]:
write_csv = []
for i in range(len(Text)):
    write_csv.append((str(i) , Text[i][:-1], int(Y[i])))

Earplugs relieve the discomfort from traveling with a cold allergy or sinus condition.


AssertionError: 

In [10]:
import csv
headers = ['id', 'Text', 'label']
with open('glove_WV.csv','w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(write_csv)

In [11]:
def get_dataset(csv_data, text_field, label_field, test=False):

    fields = [("id", None), # we won't be needing the id, so we pass in None as the field
                 ("Text", text_field), ("label", label_field)]       
    examples = []

    if test:
        # 如果为测试集，则不加载label
        for text in tqdm(csv_data['Text']):
            examples.append(data.Example.fromlist([None, text, None], fields))
    else:
        for text, label in tqdm(zip(csv_data['Text'], csv_data['label'])):
            examples.append(data.Example.fromlist([None, text, label], fields))
    return examples, fields
tokenize = lambda x: x.split()
train_data = pd.read_csv('glove_WV.csv')

TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True)
LABEL = data.Field(sequential=False, use_vocab=False)

# 得到构建Dataset所需的examples和fields
train_examples, train_fields = get_dataset(train_data, TEXT, LABEL)
# 构建Dataset数据集
train = data.Dataset(train_examples, train_fields)
train

467it [00:00, 72177.02it/s]


In [12]:
train.examples[0].Text,test.examples[0].Text

(['earplugs',
  'relieve',
  'the',
  'discomfort',
  'from',
  'traveling',
  'with',
  'a',
  'cold',
  'allergy',
  'or',
  'sinus',
  'condition.'],
 ['our',
  'water',
  'froze',
  'and',
  'batteries',
  'kept',
  'in',
  'pockets',
  'got',
  'cold.'])

In [13]:
TEXT.build_vocab(train)
vocab = TEXT.vocab
vocab.freqs.most_common(10)

[('the', 735),
 ('a', 328),
 ('of', 257),
 ('and', 238),
 ('is', 164),
 ('to', 157),
 ('in', 157),
 ('from', 121),
 ('for', 119),
 ('that', 93)]

In [14]:
cache = '.vector_cache'
if not os.path.exists(cache):
    os.mkdir(cache)
vectors = Vectors(name='/Users/denhiroshi/Downloads/glove/glove.6B.200d.txt', cache=cache)
TEXT.build_vocab(train, vectors=vectors)

In [15]:
torch.sum(TEXT.vocab.vectors != 0)/200,TEXT.vocab.vectors.shape

(tensor(2572), torch.Size([3479, 200]))

In [16]:
from torchtext.data import Iterator, BucketIterator
train_iter = Iterator(dataset = train, batch_size=64, device=-1, shuffle=False, sort_within_batch=False, repeat=False)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [17]:
weight_matrix = TEXT.vocab.vectors

In [18]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
weight_matrix = TEXT.vocab.vectors

In [19]:
class WV(nn.Module):

    def __init__(self):
        super(WV, self).__init__()
        self.word_embeddings = nn.Embedding(len(TEXT.vocab), 200)  # embedding之后的shape: torch.Size([200, 8, 300])
        self.word_embeddings.weight.data.copy_(weight_matrix)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        return embeds

In [23]:
model = WV()
emm = []
model.train()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
loss_funtion = F.cross_entropy
for step, batch_train in enumerate(train_iter):
    predicted = model(batch_train.Text)
    emm.append(predicted)
    print(predicted.shape)
len(emm)

torch.Size([53, 64, 200])
torch.Size([44, 64, 200])
torch.Size([43, 64, 200])
torch.Size([40, 64, 200])
torch.Size([39, 64, 200])
torch.Size([29, 64, 200])
torch.Size([43, 64, 200])
torch.Size([37, 19, 200])


8

In [38]:
MAX_LENGTH = 53
save = []
for temp in emm:
    for i in range(temp.shape[1]):
        now = temp[:,i,:]
        now = np.array(now.tolist())
        if now.shape[0] < MAX_LENGTH:
            add = MAX_LENGTH - now.shape[0]
            adds = np.zeros((add,200))
            now = np.concatenate((now, adds))
        save.append(now)

In [41]:
save = np.stack(save)
save.shape

(467, 53, 200)

In [ ]:
save.tofile('tensors'+str(i)+'.dat',format='%s')   